In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
#from selenium.webdriver import Chrome
#from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import time

In [7]:
def acceso(url):
    browser=uc.Chrome()
    #url="https://simple.ripley.com.pe/calzado"
    browser.get(url)
    browser.implicitly_wait(25)
    
    browser.get("https://www.falabella.com.pe/falabella-pe/collection/ver-todo-zapatos")
    browser.implicitly_wait(25)

    browser.find_element("xpath",'//*[@id="testId-accept-cookies-btn"]').click()
    browser.implicitly_wait(25)
    
    html=browser.page_source
    soup=bs(html, "html.parser")
    
    #Al terminar el proceso cierro la pestaña
    browser.close()
    #Cierro el navegador
    browser.quit()
    
    return soup

In [3]:
def next_page(enlace,pagina):
    enlace=enlace+"?page={}"
    enlace=enlace.format(pagina)
    return enlace

In [4]:
#Extraer texto
def text_extract(textos):
    lista=[]
    for texto in textos:
        lista.append(texto.text)
    return lista

#Extraer precio
def prices_extract(precios):
    lista=[]
    for precio in precios:
        lista.append(precio.text.strip()[3:])
    return lista

In [8]:
url="https://www.falabella.com.pe/falabella-pe/collection/ver-todo-zapatos"
items_categoria=[]
marcas_categoria=[]
precios_categoria=[]


for i in range(1,8):
    #Genera el enlace de la página i
    url_page=next_page(url,i)
    #Obtiene el html de la página i
    soup=acceso(url_page)
    
    #Encuentra los items, marcas y precios en el html de la página i
    items=soup.find_all("b",{"class":"jsx-184544934 copy2 primary jsx-3451706699 normal line-clamp line-clamp-3 pod-subTitle subTitle-rebrand"})
    marcas=soup.find_all("b", {"class":"jsx-184544934 title1 secondary jsx-3451706699 bold pod-title title-rebrand"})
    precios=soup.find_all("span", {"class":"copy10 primary medium jsx-3451706699 normal line-height-22"})
    
    #Extra los items, marcas y precios de la página i
    items_list=text_extract(items)
    marcas_list=text_extract(marcas)
    precios_list=prices_extract(precios)
    
    #Coloca todos los elementos en una listas globales
    items_categoria.append(items_list)
    marcas_categoria.append(marcas_list)
    precios_categoria.append(precios_list)
    
    
#Como nuestro resultado será una lista de listas
#La siguiente función las coloca en una sola
#Ejemplo A=[[1,2], [3,4],[5,6]]
#Retorna A=[1,2,3,4,5,6]

def unir_listas(lista_de_listas):
    """
    Convierte una lista de listas en una sola lista usando un bucle.
    
    :param lista_de_listas: Lista de listas a unir.
    :return: Lista unificada.
    """
    resultado = []
    for sublista in lista_de_listas:
        resultado.extend(sublista)
    return resultado


items_categoria=unir_listas(items_categoria)
marcas_categoria=unir_listas(marcas_categoria)
precios_categoria=unir_listas(precios_categoria)
    
df=pd.DataFrame({
    "Item":items_categoria,
    "Marca":marcas_categoria,
    "Precio":precios_categoria
    })
df["categoria"]="Calzado"
df


,Item,Marca,Precio,categoria
0,Zapatillas Deportivas Running Mujer Runfalcon ...,ADIDAS,199,Calzado
1,Zapatillas Urbanas Mujer Bobs Buno Blanco,SKECHERS,189,Calzado
2,Zapatillas Running Mujer Soluna Negro,DIADORA,159.92,Calzado
3,Sandalia Casual Mujer Belina Do,STEFANO COCCI,59.90,Calzado
4,Zapato Ballerina Mujer Asticz,SYBILLA,69.93,Calzado
...,...,...,...,...
331,Alpargata Mujer Gernan,BASEMENT,149.90,Calzado
332,Zapato Ballerina Mujer Abaletahan,SYBILLA,79.90,Calzado
333,Pantufla Mujer,SYBILLA,29.94,Calzado
334,Alpargata Mujer,REDWOOD,19.90,Calzado


In [12]:
df["Item"].head(1)

0    Zapatillas Deportivas Running Mujer Runfalcon ...
Name: Item, dtype: object